In [58]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import datetime

%load_ext tensorboard 


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [59]:
#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_ds, test_ds),ds_info = tfds.load('cifar10', split=['train', 'test'], as_supervised=True, with_info=True)


In [60]:
#train_images, test_images = train_images / 255.0, test_images / 255.0
def prepare_data(data): 
    #data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
    data = data.map(lambda img, target: ((img/255), target))
    data = data.map(lambda img, target: (img, tf.one_hot(target, depth=10))) # no one-hot encoding
    data = data.cache()
    data = data.shuffle(1000)
    data = data.batch(32)
    data = data.prefetch(20)
    return data

In [61]:
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [62]:
train_dataset = train_ds.apply(prepare_data)
test_dataset = test_ds.apply(prepare_data)

In [63]:
#plt.figure(figsize=(10,10))
#for i in range(25):
#    plt.subplot(5,5,i+1)
#    plt.xticks([])
#    plt.yticks([])
#    plt.grid(False)
#    plt.imshow(train_images[i])
#    # The CIFAR labels happen to be arrays, 
#    # which is why you need the extra index
#    plt.xlabel(classes[train_labels[i][0]])
#plt.show()
#tfds.show_examples(train_ds, ds_info)

In [64]:
train_iterator = train_dataset.as_numpy_iterator().next()
print(train_iterator[0].shape)

(32, 32, 32, 3)


2022-12-04 16:56:40.960590: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-12-04 16:56:40.960755: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [65]:
class CNN(tf.keras.Model):

    # 1. constructor
    def __init__(self):
        super().__init__()
        # inherit functionality from parent class

        # optimizer, loss function and metrics
        self.metrics_list = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), tf.keras.metrics.Mean(name="loss")]
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
        
        self.loss_function = tf.keras.losses.CategoricalCrossentropy()

        # layers to encode the images (both layers used for both images)
        self.convlayer1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=( 32,32,3))
        self.convlayer2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")
        #self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)


        #self.dense1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        #self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        #self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.global_pool = tf.keras.layers.GlobalAvgPool2D()
        
        self.out_layer = tf.keras.layers.Dense(10,activation=tf.nn.softmax)
        
    # 2. call method (forward computation)
    def call(self, image, training=False):

        x = image
        for layer in self.layers:
            x = layer(x)

        return x


    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model

    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    # 5. train step method
    @tf.function
    def train_step(self, data):
        x,y = data

        with tf.GradientTape() as tape:
            output = self(x, training=True)
            loss = self.loss_function(y, output)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update the state of the metrics according to loss
        self.metrics[0].update_state(y, output)
        self.metrics[1].update_state(loss)
        
        # return a dictionary with metric names as keys and metric results as values
        return {m.name : m.result() for m in self.metrics}

    # 6. test_step method
    @tf.function
    def test_step(self, data):
        x, y = data
        # same as train step (without parameter updates)
        output = self(x, training=False)
        loss = self.loss_function(y, output)
        self.metrics[0].update_state(y, output)
        self.metrics[1].update_state(loss)
        
        return {m.name : m.result() for m in self.metrics}

In [66]:
def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer



In [67]:
import tqdm
def training_loop(model, 
                train_ds, 
                test_ds, 
                start_epoch,
                epochs,
                train_summary_writer, 
                val_summary_writer, 
                save_path):

    # 1. iterate over epochs
    for e in range(start_epoch, epochs):

        # 2. train steps on all batches in the training data
        for data in tqdm.tqdm(train_ds, position=0, leave=True):
            print(data)
            metrics = model.train_step(data) # we save to metrics because we want to print it and tensorboard

        ## Training data
        # 3. log and print training metrics
        with train_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)
            # alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
        
        #print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()])
        
        # 4. reset metric objects for next epch
        model.reset_metrics()

        #####################################################

        ## Validation data
        # 5. evaluate on validation data
        for data in test_ds:
            metrics = model.test_step(data)
        
        # 6. log validation metrics
        with val_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)
            # alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
            
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])
        # 7. reset metric objects
        model.reset_metrics()
        
    # 8. save model weights if save_path is given
    if save_path:
        model.save_weights(save_path)

In [68]:
# 1. instantiate model
cnn = CNN()

# 2. choose a path to save the weights
save_path_subtask = "RUN4"

train_summary_writer, val_summary_writer = create_summary_writers(config_name="RUN4")


# 3. pass arguments to training loop function
training_loop(model=cnn,
    train_ds=train_ds,
    test_ds=test_ds,
    start_epoch=0,
    epochs=10,
    train_summary_writer=train_summary_writer,
    val_summary_writer=val_summary_writer,
    save_path=save_path_subtask)

  0%|          | 0/50000 [00:00<?, ?it/s]

(<tf.Tensor: shape=(32, 32, 3), dtype=uint8, numpy=
array([[[143,  96,  70],
        [141,  96,  72],
        [135,  93,  72],
        ...,
        [ 96,  37,  19],
        [105,  42,  18],
        [104,  38,  20]],

       [[128,  98,  92],
        [146, 118, 112],
        [170, 145, 138],
        ...,
        [108,  45,  26],
        [112,  44,  24],
        [112,  41,  22]],

       [[ 93,  69,  75],
        [118,  96, 101],
        [179, 160, 162],
        ...,
        [128,  68,  47],
        [125,  61,  42],
        [122,  59,  39]],

       ...,

       [[187, 150, 123],
        [184, 148, 123],
        [179, 142, 121],
        ...,
        [198, 163, 132],
        [201, 166, 135],
        [207, 174, 143]],

       [[187, 150, 117],
        [181, 143, 115],
        [175, 136, 113],
        ...,
        [201, 164, 132],
        [205, 168, 135],
        [207, 171, 139]],

       [[195, 161, 126],
        [187, 153, 123],
        [186, 151, 128],
        ...,
        [212, 177, 147

ValueError: in user code:

    File "/var/folders/2q/hwnn9141093b7bkbnd4bm7tc0000gn/T/ipykernel_12063/1901598850.py", line 55, in train_step  *
        output = self(x, training=True)
    File "/Users/leonackermann/miniforge3/envs/iannwtf/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/2q/hwnn9141093b7bkbnd4bm7tc0000gn/T/__autograph_generated_filenxmbfpf6.py", line 24, in tf__call
        ag__.for_stmt(ag__.ld(self).layers, None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'layer'})
    File "/var/folders/2q/hwnn9141093b7bkbnd4bm7tc0000gn/T/__autograph_generated_filenxmbfpf6.py", line 22, in loop_body
        x = ag__.converted_call(ag__.ld(layer), (ag__.ld(x),), None, fscope)

    ValueError: Exception encountered when calling layer "cnn_8" "                 f"(type CNN).
    
    in user code:
    
        File "/var/folders/2q/hwnn9141093b7bkbnd4bm7

In [ ]:
cnn.summary()

Model: "cnn_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           multiple                  0 (unused)
                                                                 
 conv2d_9 (Conv2D)           multiple                  0 (unused)
                                                                 
 max_pooling2d_4 (MaxPooling  multiple                 0 (unused)
 2D)                                                             
                                                                 
 dense_16 (Dense)            multiple                  0 (unused)
                                                                 
 dense_17 (Dense)            multiple                  0 (unused)
                                                                 
 dense_18 (Dense)            multiple                  0 (unused)
                                                             

In [ ]:
# open the tensorboard logs
%tensorboard --logdir logs/